In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorize_functions
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.datasets import make_classification


Einladen der vektorisierten Daten

In [1]:
%run ../../functions/vectorize_functions.py

In [2]:
filepath_name = (('../../../data/twitter_hate-speech/train_cleaned.csv'))
df_cleaned = pd.read_csv(filepath_name, encoding='utf-8')

In [5]:
X_train, X_test, y_train, y_test = vectorize_tfidf(df=df_cleaned, text_column='tweet_cleaned', 
                                                                                 label_column="label")

In [6]:
positive = np.count_nonzero(y_train == 1)
negative = np.count_nonzero(y_train == 0)
print("Train")
print("- Positive:", positive)
print("- Negative:", negative)
print("- Verhältnis:", negative / positive)

positive = np.count_nonzero(y_test == 1)
negative = np.count_nonzero(y_test == 0)
print("Test")
print("- Positive:", positive)
print("- Negative:", negative)
print("- Verhältnis:", negative / positive)

Train
- Positive: 930
- Negative: 12807
- Verhältnis: 13.770967741935484
Test
- Positive: 374
- Negative: 5514
- Verhältnis: 14.743315508021391


In [7]:
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)

print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (13737, 4925)
y_train shape (13737,)
X_test shape (5888, 4925)
y_test shape (5888,)


# Evaluation

# SVM


In [11]:
results_list = []

param_grid = {
    'kernel': ['sigmoid'],
    #'degree': [3, 4, 5],
    'C': [0.01, 0.1, 1],
    'gamma': [0.01, 0.1, 1],
    'random_state': [40, 42, 44],
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

svm = SVC()

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

for params in grid_search.cv_results_['params']:
    model = SVC(**params)  
    model.fit(X_train, y_train)  

    y_train_pred = model.predict(X_train)

    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    result_dict = {
        'vectorization': 'tfidf',
        'kernel': params['kernel'],
        'degree': 'X',
        'C': params['C'],
        'gamma': params['gamma'],
        'class_weight': params['class_weight'],
        'train_accuracy': train_accuracy,
        'train_recall': train_recall,
        'train_precision': train_precision,
        'train_f1': train_f1,
        'test_accuracy': test_accuracy,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_f1': test_f1
    }

    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)

results_df.to_csv('svm_grid_tfidf_sigmoid.csv', index=False)

print(results_df)


Fitting 3 folds for each of 81 candidates, totalling 243 fits


c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Pr

   vectorization   kernel degree     C  gamma  class_weight  train_accuracy  \
0          tfidf  sigmoid      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
1          tfidf  sigmoid      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
2          tfidf  sigmoid      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
3          tfidf  sigmoid      X  0.01   0.10  {0: 1, 1: 1}        0.932300   
4          tfidf  sigmoid      X  0.01   0.10  {0: 1, 1: 1}        0.932300   
..           ...      ...    ...   ...    ...           ...             ...   
76         tfidf  sigmoid      X  1.00   0.10  {0: 1, 1: 5}        0.961928   
77         tfidf  sigmoid      X  1.00   0.10  {0: 1, 1: 5}        0.961928   
78         tfidf  sigmoid      X  1.00   1.00  {0: 1, 1: 5}        0.949916   
79         tfidf  sigmoid      X  1.00   1.00  {0: 1, 1: 5}        0.949916   
80         tfidf  sigmoid      X  1.00   1.00  {0: 1, 1: 5}        0.949916   

    train_recall  train_precision  train_f1  test_a

In [12]:
results_list = []

param_grid = {
    'kernel': ['linear'],
    #'degree': [3, 4, 5],
    'C': [0.01, 0.1, 1],
    'gamma': [0.01, 0.1, 1],
    'random_state': [40, 42, 44],
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

svm = SVC()

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

for params in grid_search.cv_results_['params']:
    model = SVC(**params)  
    model.fit(X_train, y_train)  

    y_train_pred = model.predict(X_train)

    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    result_dict = {
        'vectorization': 'tfidf',
        'kernel': params['kernel'],
        'degree': 'X',
        'C': params['C'],
        'gamma': params['gamma'],
        'class_weight': params['class_weight'],
        'train_accuracy': train_accuracy,
        'train_recall': train_recall,
        'train_precision': train_precision,
        'train_f1': train_f1,
        'test_accuracy': test_accuracy,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_f1': test_f1
    }

    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)

results_df.to_csv('svm_grid_tfidf_linear.csv', index=False)

print(results_df)


Fitting 3 folds for each of 81 candidates, totalling 243 fits


c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Pr

   vectorization  kernel degree     C  gamma  class_weight  train_accuracy  \
0          tfidf  linear      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
1          tfidf  linear      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
2          tfidf  linear      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
3          tfidf  linear      X  0.01   0.10  {0: 1, 1: 1}        0.932300   
4          tfidf  linear      X  0.01   0.10  {0: 1, 1: 1}        0.932300   
..           ...     ...    ...   ...    ...           ...             ...   
76         tfidf  linear      X  1.00   0.10  {0: 1, 1: 5}        0.978525   
77         tfidf  linear      X  1.00   0.10  {0: 1, 1: 5}        0.978525   
78         tfidf  linear      X  1.00   1.00  {0: 1, 1: 5}        0.978525   
79         tfidf  linear      X  1.00   1.00  {0: 1, 1: 5}        0.978525   
80         tfidf  linear      X  1.00   1.00  {0: 1, 1: 5}        0.978525   

    train_recall  train_precision  train_f1  test_accuracy  tes

In [13]:
results_list = []

param_grid = {
    'kernel': ['poly'],
    'degree': [3, 4, 5],
    'C': [0.01, 0.1, 1],
    'gamma': [0.01, 0.1, 1],
    'random_state': [40, 42, 44],
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 3}]
}

svm = SVC()

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

for params in grid_search.cv_results_['params']:
    model = SVC(**params)  
    model.fit(X_train, y_train)  

    y_train_pred = model.predict(X_train)

    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    result_dict = {
        'vectorization': 'tfidf',
        'kernel': params['kernel'],
        'degree': params['degree'],
        'C': params['C'],
        'gamma': params['gamma'],
        'class_weight': params['class_weight'],
        'train_accuracy': train_accuracy,
        'train_recall': train_recall,
        'train_precision': train_precision,
        'train_f1': train_f1,
        'test_accuracy': test_accuracy,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_f1': test_f1
    }

    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)

results_df.to_csv('svm_grid_tfidf_poly.csv', index=False)


print(results_df)


Fitting 3 folds for each of 243 candidates, totalling 729 fits


c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Pr

    vectorization kernel  degree     C  gamma  class_weight  train_accuracy  \
0           tfidf   poly       3  0.01   0.01  {0: 1, 1: 1}        0.932300   
1           tfidf   poly       3  0.01   0.01  {0: 1, 1: 1}        0.932300   
2           tfidf   poly       3  0.01   0.01  {0: 1, 1: 1}        0.932300   
3           tfidf   poly       3  0.01   0.10  {0: 1, 1: 1}        0.932300   
4           tfidf   poly       3  0.01   0.10  {0: 1, 1: 1}        0.932300   
..            ...    ...     ...   ...    ...           ...             ...   
238         tfidf   poly       5  1.00   0.10  {0: 1, 1: 3}        0.932300   
239         tfidf   poly       5  1.00   0.10  {0: 1, 1: 3}        0.932300   
240         tfidf   poly       5  1.00   1.00  {0: 1, 1: 3}        0.999927   
241         tfidf   poly       5  1.00   1.00  {0: 1, 1: 3}        0.999927   
242         tfidf   poly       5  1.00   1.00  {0: 1, 1: 3}        0.999927   

     train_recall  train_precision  train_f1  test_

In [14]:
results_list = []

param_grid = {
    'kernel': ['rbf'],
    #'degree': [],
    'C': [0.01, 0.1, 1],
    'gamma': [0.01, 0.1, 1],
    'random_state': [40, 42, 44],
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 3}]
}

svm = SVC()

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='f1', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

for params in grid_search.cv_results_['params']:
    model = SVC(**params)  
    model.fit(X_train, y_train)  

    y_train_pred = model.predict(X_train)

    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)

    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    result_dict = {
        'vectorization': 'tfidf',
        'kernel': params['kernel'],
        'degree': 'X',
        'C': params['C'],
        'gamma': params['gamma'],
        'class_weight': params['class_weight'],
        'train_accuracy': train_accuracy,
        'train_recall': train_recall,
        'train_precision': train_precision,
        'train_f1': train_f1,
        'test_accuracy': test_accuracy,
        'test_recall': test_recall,
        'test_precision': test_precision,
        'test_f1': test_f1
    }

    results_list.append(result_dict)

results_df = pd.DataFrame(results_list)

results_df.to_csv('svm_grid_tfidf_rbf.csv', index=False)

print(results_df)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Pr

   vectorization kernel degree     C  gamma  class_weight  train_accuracy  \
0          tfidf    rbf      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
1          tfidf    rbf      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
2          tfidf    rbf      X  0.01   0.01  {0: 1, 1: 1}        0.932300   
3          tfidf    rbf      X  0.01   0.10  {0: 1, 1: 1}        0.932300   
4          tfidf    rbf      X  0.01   0.10  {0: 1, 1: 1}        0.932300   
..           ...    ...    ...   ...    ...           ...             ...   
76         tfidf    rbf      X  1.00   0.10  {0: 1, 1: 3}        0.963529   
77         tfidf    rbf      X  1.00   0.10  {0: 1, 1: 3}        0.963529   
78         tfidf    rbf      X  1.00   1.00  {0: 1, 1: 3}        0.999490   
79         tfidf    rbf      X  1.00   1.00  {0: 1, 1: 3}        0.999490   
80         tfidf    rbf      X  1.00   1.00  {0: 1, 1: 3}        0.999490   

    train_recall  train_precision  train_f1  test_accuracy  test_recall  \
